<a href="https://colab.research.google.com/github/cytoscape/cytoscape-automation/blob/master/for-scripters/R/colab/Cytoscape-and-NDEx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cytoscape and NDEx
## Alexander Pico, Florian Auer


*The R markdown is available. [download the Rmd from GitHub](https://raw.githubusercontent.com/cytoscape/cytoscape-automation/master/for-scripters/R/notebooks/Cytoscape-and-NDEx.Rmd).*

<hr />

*Cytoscape* is a well-known bioinformatics tool for displaying and exploring biological networks.
**R** is a powerful programming language and environment for statistical and exploratory data analysis.
*RCy3* uses CyREST to communicate between **R** and Cytoscape, allowing any graphs (e.g., igraph, graphNEL or dataframes) to be viewed, explored and manipulated with the Cytoscape point-and-click visual interface. Thus, via RCy3, these two quite different, quite useful bioinformatics software environments are connected, mutually enhancing each other, providing new possibilities for exploring biological data.

# Installation

In [ ]:
if(!"RCy3" %in% installed.packages()){
    install.packages("BiocManager")
    BiocManager::install("RCy3")
}
library(RCy3)

if(!"ndexr" %in% installed.packages()){
    install.packages("BiocManager")
    BiocManager::install("ndexr")
}
library(ndexr)

If you are using a remote notebook environment such as Google Colab, please execute the cell below. (If you're running on your local notebook, you don't need to do that.)

In [ ]:
# Connect to local Cytoscape
browserClientJs <- getBrowserClientJs()
IRdisplay::display_javascript(browserClientJs)

# Prerequisites
In addition to these packages, you will need:

  * **Cytoscape 3.6.1** or greater, which can be downloaded from http://www.cytoscape.org/download.php. Simply follow the installation instructions on screen.

# Finding networks
The Network Data Exchange (NDEx) is a platform for network storage, retrieval and exchange. Developed in close collaboration with Cytoscape, it is a natural partner for Cytoscape network queries and management.  

You can start with NDEx by first establishing a connection.

In [ ]:
ndexcon <- ndex_connect()

We will use **ndexcon** throughout the other ndexr calls. For example, a basic search.

In [ ]:
networks <- ndex_find_networks(ndexcon, "Breast Cancer")
print(networks[,c("name","externalId","nodeCount","edgeCount")])

That *print* statement provides a nifty way to browse the search results. You'll notice that we got results that hit each of the search terms individually, thus including any pathway with "cancer" in the name. That's perhaps a bit too broad...

In [ ]:
networks <- ndex_find_networks(ndexcon, "BRCA")
print(networks[,c("name","externalId","nodeCount","edgeCount")])

Ok. We can work with this list. Let's use the first hit. **Note: you are going to get different hits as this database changes over time, so proceed with any hit you like.**

In [ ]:
networkId = networks$externalId[1]
network = ndex_get_network(ndexcon, networkId)
print(network)

# Viewing networks
If we want to visualize and interact with this network, Cytoscape is our best choice. Both NDEx and Cytoscape work with the CX network data format, so the import can directly be managed by the CyNDEx component of Cytoscape, supported by RCy3...

In [ ]:
brca.net.suid <- importNetworkFromNDEx(networkId)

![](https://cytoscape.github.io/cytoscape-automation/for-scripters/R/notebooks/data/img/NDEx.png)
**Note: Do not use the "rcx" frunctions provided by ndexr to pass through igraph in order to import into Cytoscape. This unnecessary intermediate step will result in scientific notation values being translated to type "other" (denoted by "x") and ultimately converted to Strings in Cytoscape. All style information is lost as well.**

# Sending networks to NDEx
Now say you have a network in Cytoscape and you want to send it to NDEx for storage, sharing or exchange. First, you need to have an NDEx account. Go to http://www.ndexbio.org to create one if you haven't already. You will need your username and password coming up soon.

But first, let's get a fresh network into Cytoscape. This could be any network, we're just using the demo igraph network as an example.

In [ ]:
ig2 <- makeSimpleIgraph()
net.suid <- createNetworkFromIgraph(ig2, 'Simple Network', 'Simple Collection')

And, now we send it off to NDEx.

In [ ]:
user <- "your_NDEx_account_username"  #replace with your info
pass <- "your_NDEx_account_password"  #replace with your info
exportNetworkToNDEx(user, pass, isPublic=TRUE, network=net.suid)

Voila! Now anyone can search or browse to find your network and then use it in their research together with the full history and attribution of your contributions to the network.